In [4]:
import plyj.parser as plyj


In [5]:
parser = plyj.Parser()


Generating LALR tables


In [18]:
tree = parser.parse_file(('helloworld.java'))
print(tree)

CompilationUnit(package_declaration=None, import_declarations=[], type_declarations=[ClassDeclaration(name='helloworld', body=[MethodDeclaration(name='main', modifiers=['public', 'static'], type_parameters=[], parameters=[FormalParameter(variable=Variable(name='args', dimensions=0), type=Type(name=Name(value='String'), type_arguments=[], enclosed_in=None, dimensions=1), modifiers=[], vararg=False)], return_type='void', body=[ExpressionStatement(expression=MethodInvocation(name='println', arguments=[Literal(value='"Hello, World"')], type_arguments=[], target=Name(value='System.out')))], abstract=False, extended_dims=0, throws=None)], modifiers=['public'], type_parameters=[], extends=None, implements=[])])


In [16]:
tree = parser.parse_string('class helloworld { }')
print(tree)

CompilationUnit(package_declaration=None, import_declarations=[], type_declarations=[ClassDeclaration(name='helloworld', body=[], modifiers=[], type_parameters=[], extends=None, implements=[])])


In [17]:
tree = parser.parse_expression('1 / 2 * (float) 3')
print(tree)

Multiplicative(operator='*', lhs=Multiplicative(operator='/', lhs=Literal(value='1'), rhs=Literal(value='2')), rhs=Cast(target=Type(name='float', type_arguments=[], enclosed_in=None, dimensions=0), expression=Literal(value='3')))


In [15]:
print(tree)

Multiplicative(operator='*', lhs=Multiplicative(operator='/', lhs=Literal(value='1'), rhs=Literal(value='2')), rhs=Cast(target=Type(name='float', type_arguments=[], enclosed_in=None, dimensions=0), expression=Literal(value='3')))


In [20]:
import unittest

import plyj.parser as plyj
import plyj.model as model

one = model.Literal('1')
two = model.Literal('2')
three = model.Literal('3')
a = model.Name('a')
b = model.Name('b')
c = model.Name('c')
d = model.Name('d')
e = model.Name('e')

def bin(operator, operand1, operand2):
    return model.BinaryExpression(operator, operand1, operand2)

def u(operator, operand):
    return model.Unary(operator, operand)

expression_tests = [
    # simple test for each operator
    ('1+2', bin('+', one, two)),
    (' 1 + 2 ', bin('+', one, two)),
    ('1-2', bin('-', one, two)),
    ('1*2', bin('*', one, two)),
    ('1/2', bin('/', one, two)),
    ('1%2', bin('%', one, two)),
    ('1^2', bin('^', one, two)),
    ('1&2', bin('&', one, two)),
    ('1&&2', bin('&&', one, two)),
    ('1|2', bin('|', one, two)),
    ('1||2', bin('||', one, two)),
    ('1==2', bin('==', one, two)),
    ('1!=2', bin('!=', one, two)),
    ('1<2', bin('<', one, two)),
    ('1<=2', bin('<=', one, two)),
    ('1>2', bin('>', one, two)),
    ('1>=2', bin('>=', one, two)),
    ('1<<2', bin('<<', one, two)),
    ('1>>2', bin('>>', one, two)),
    ('1>>>2', bin('>>>', one, two)),
    # left associativity
    ('1+2+3', bin('+', bin('+', one, two), three)),
    # precedence
    ('1+2*3', bin('+', one, bin('*', two, three))),
    # parenthesized expressions
    ('(1+2)*3', bin('*', bin('+', one, two), three)),
    # conditionals
    ('a ? b : c', model.Conditional(a, b, c)),
    ('a ? b ? c : d : e', model.Conditional(a, model.Conditional(b, c, d), e)),
    ('a ? b : c ? d : e', model.Conditional(a, b, model.Conditional(c, d, e))),
    # unary expressions
    ('+a', u('+', a)),
    ('-a', u('-', a)),
    ('!a', u('!', a)),
    ('!!a', u('!', u('!', a))),
    ('~a', u('~', a)),
    ('++a', u('++x', a)),
    ('--a', u('--x', a)),
    ('a++', u('x++', a)),
    ('a--', u('x--', a)),
    # assignment expressions
    ('a = 1', model.Assignment('=', a, one)),
    ('a += 1', model.Assignment('+=', a, one)),
    ('a -= 1', model.Assignment('-=', a, one)),
    ('a *= 1', model.Assignment('*=', a, one)),
    ('a /= 1', model.Assignment('/=', a, one)),
    ('a %= 1', model.Assignment('%=', a, one)),
    ('a ^= 1', model.Assignment('^=', a, one)),
    ('a &= 1', model.Assignment('&=', a, one)),
    ('a |= 1', model.Assignment('|=', a, one)),
    ('a <<= 1', model.Assignment('<<=', a, one)),
    ('a >>= 1', model.Assignment('>>=', a, one)),
    ('a >>>= 1', model.Assignment('>>>=', a, one)),
    # casts
    ('(Foo) a', model.Cast(model.Type(model.Name('Foo')), a)),
    ('(int[]) a', model.Cast(model.Type('int', dimensions=1), a)),
    ('(Foo[]) a', model.Cast(model.Type(model.Name('Foo'), dimensions=1), a)),
    ('(Foo<T>) a', model.Cast(model.Type(model.Name('Foo'), type_arguments=[model.Type(model.Name('T'))]), a)),
    ('(Foo<T>.Bar) a', model.Cast(model.Type(model.Name('Bar'),
                                  enclosed_in=model.Type(model.Name('Foo'), type_arguments=[model.Type(model.Name('T'))])), a)),
    # method invocation
    ('foo.bar()', model.MethodInvocation(name='bar', target=model.Name(value='foo'))),
    ('foo.class.getName()', model.MethodInvocation(target=model.ClassLiteral(model.Type(model.Name('foo'))), name='getName')),

    ('foo.Class[].class', model.ClassLiteral(model.Type(model.Name('foo.Class'), dimensions=1)))
]

class ExpressionTest(unittest.TestCase):

    def setUp(self):
        self.parser = plyj.Parser()

    def test_expressions(self):
        for expr, result in expression_tests:
            t = self.parser.parse_expression(expr)
            self.assertEqual(t, result, 'for {} got: {}, expected: {}'.format(expr, t, result))

if __name__ == '__main__':
    unittest.main()

E
ERROR: C:\Users\semal\AppData\Roaming\jupyter\runtime\kernel-46785ad4-9fa7-48e4-b72b-320c0ee6d3d3 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\semal\AppData\Roaming\jupyter\runtime\kernel-46785ad4-9fa7-48e4-b72b-320c0ee6d3d3'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

C:\Users\semal\Anaconda3.3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [21]:
%tb


SystemExit: True

In [28]:
tree2 = parser.parse_file(('WordCount.java'))
for element in tree2:
    print(element)

TypeError: 'CompilationUnit' object is not iterable

In [24]:
print(type(tree2))

<class 'plyj.model.CompilationUnit'>


In [25]:
for path, node in tree2:
...     print (path, node)

TypeError: 'CompilationUnit' object is not iterable

In [27]:
import sys
print('''usage: parse_expr.py <expression> ...
   Example: parse_expr.py '1+2' '3' 'j = (int) i + 3' ''')

import plyj.parser as plyj

parser = plyj.Parser()
for expr in sys.argv[1:]:
    print(parser.parse_expression(expr))

usage: parse_expr.py <expression> ...
   Example: parse_expr.py '1+2' '3' 'j = (int) i + 3' 


Generating LALR tables


Unary(sign='-', expression=Name(value='f'))
error: LexToken(:,':',1,3)
Illegal character '\' (0x5c) in line 1
Illegal character '\' (0x5c) in line 1
Illegal character '\' (0x5c) in line 1
Illegal character '\' (0x5c) in line 1
Illegal character '\' (0x5c) in line 1
Illegal character '\' (0x5c) in line 1
Illegal character '\' (0x5c) in line 1
None
